In [26]:
import nltk
from nltk.collocations import *
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords as nltk_stopwords
from string import punctuation
import requests
import re

# Endereço da API dos discursos
BABEL_API_URL = 'https://babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=1'

# Pegando os 200 primeiros discursos da API do Babel
speeches = []
data = requests.get(BABEL_API_URL).json()['results']
for i in range(2, 10):
    data += requests.get(BABEL_API_URL + '&page=%s' % i).json()['results']
    
for speech in data:
    for attr in speech['attrs']:
        if attr['field'] == 'original':
            speeches.append(attr['value'])
            break

In [27]:
# Criando a lista de stopwords em português
EXTRA_STOPWORDS = [
    'sr.', 'nesse', 'deputados', '``', "''", 'empresa', 'trabalhadores', 'tema', 'brasil', 'brasileiro',
    'brasileira', 'sociedade', 'grandes', 'meios', 'principal', 'deputada', 'nesta', 'valor', 'reais',
    'representante', 'brasileiros', 'necessidade', 'quero', 'ser', 'geral', 'todo', 'toda', 'estar',
    'ter', 'parlamentares', 'região', 'forma', 'parte', 'disso', 'debate', 'entregar', 'nessa', 'nome',
    'vez', 'quer', 'primeira', 'soberania', 'justamente', 'ponto', 'presentes', 'faz', 'futuro', 'podem',
    'maneira', 'falar', 'interesses', 'caso', 'espaço', 'entrega', 'deste', 'mesma', 'negócio', 'existe',
    'avançar', 'ano', 'diz', 'próprios', 'criação', 'própria', 'dizendo', 'trazer', 'preocupação', 'ali',
    'ficar', 'desse', 'importância', 'senhor', 'queremos', 'poderia', 'ir', 'próprio', 'área', 'segundo',
    'acontece', 'sei', 'sabem', 'comum', 'mim', 'tratar', 'obrigação', 'falando', 'discurso', 'muitas',
    'acabou', 'falou', 'outro', 'capacidade', 'força','querem', 'significa', 'serviço', 'dados', 'tentar',
    'ninguém', 'gente', 'ideia', 'deputadas', 'dentro', 'fazendo', 'mão', 'época', 'uso', 'fiz', 'último',
    'nenhum', 'números', 'alguma', 'acreditamos', 'achamos', 'pagar', 'paga', 'conjunto', 'contentam',
    'lacunas', 'esperamos', 'digo', 'sente', 'logo', 'pesquisar', 'principais', 'mudar', 'sequer', 'pagou',
    'pagando', 'traz', 'desafio', 'concreto', 'atende', 'tendo', 'aberto', 'curto', 'recebe', 'receber',
    'décadas', 'minutos', 'horas', 'segundos', 'motivo', 'posso', 'dando', 'entra', 'volto', 'construir',
    'algumas', 'passar', 'muita', 'nisso', 'deveria', 'dá', 'construir', 'muito', 'longo', 'muitas', 'outras',
    'existem', 'conseguimos', 'precisamos', 'feita', 'mencionou', 'falei', 'cujo', 'hora', 'dizia', 'venha',
    'conseguindo', 'conseguir', 'votar', 'alguém', 'somente', 'todas', 'fizemos', 'citar', 'saiba', 'boa',
    'deveriam', 'acontecendo', 'algo', 'apresentei', 'sabemos', 'outra', 'junto', 'daqui', 'aconteceu',
    'haver', 'sinto', 'preciso', 'muitos', 'minimamente', 'amanhã', 'ontem', 'partir', 'objetivo', 'opinião',
    'vivemos', 'discutindo', 'agradecemos', 'utilizados', 'contra', 'dona', 'falta', 'possa', 'manhã', 'novas',
    'após', 'pessoas', 'fundamental', 'desses', 'devido', 'item', 'século', 'domingo', 'sábado',
    'presidente', ',', '.', '...', 'é', 'questão', 'art', 'ordem', 'v.exa', ':', 'governo', 'sr', 'agência',
    'aqui', 'vai', 'artigo', '§', 'neste', 'vamos', 'agora', "''", 'fazer', 'mesa', 'ainda', 'porque', 'trata',
    'estrutura', 'sobre', 'então', 'todos', 'obstrução', 'votação', 'presença', 'deputados', 'vou', 'brasil',
    'discutir', 'vigência', 'colocar', 'regimento', 'momento', ';', 'dois', 'dessa', 'medida', 'proposta',
    'casa', 'matéria', 'queria', 'assim', 'possamos', 'microfone', 'certeza', 'hoje', 'profissional', 'deixar',
    'provisória', 'ora', 'base', 'importante', 'veto', 'fala', '!', 'aumento', 'inciso', 'sra.', 'talvez',
    'cima', 'servir', 'nunca', 'dias', 'deus', 'dei', 'entendemos', 'chega', 'possam', 'entendo', 'poderá',
    'celeridade', 'tirar', 'mista', 'fechou-se', 'lado', 'lido', 'repassado', 'demais', 'venho', 'marcar',
    'xiii', 'diálogo', 'podemos', 'apenas', 'poder', 'efeitos', 'pode', 'acordo', 'solicitação',
    'reflexão', '?', 'ausência', 'aprovada', 'lideranças', 'dizer', 'portanto', 'peço', 'recolher',
    'prática', 'pois', 'milhões', 'bilhões', 'melhoria', 'atividade', 'claro', 'saber', 'dar',
    'avanço', 'condições', 'desastre', 'especialmente', 'exatamente', 'política', 'vezes', 'fazê-lo', 'têm',
    'derrubar', 'precisa', 'custo', 'necessária', 'cláusula', 'proposição', '-', 'palavra', 'tempo', 'segundos',
    'fez', 'necessário', 'zero', 'interesse', 'srs', 'sr', 'sras', 'sra', 'deputado', 'presidente', 'é', 'nº',
    's.a.', 'v.exa.', 'v.exa', '#', 'anos', 'º', 'exa', 'mesa', 'legislatura', 'sessão', 'maioria', 'seguinte',
    'mandato', 'bilhões', 'quilômetros', 'ª', 'parabéns', 'membros', 'convido', 'usual', 'biênio',
    'brasil', 'palavra', 'discussão', 'período', 'início', 'pronunciamento', 'suplente', 'atividade', 'ação',
    'ações', 'daqueles', 'diferenças', 'pasta', 'milhares', 'srªs', 'emenda', 'àqueles', 'tamanha', 'mês',
    'capaz', 'km', 'modelo', 'tarefas', 'colegas', 'programa', 'voz', 'pronunciamento',
    'casa', 'sessão', 'deliberativa', 'solene', 'ordinária', 'extraordinária', 'encaminhado', 'orador',
    'divulgar', 'deputado', 'parlamentar', 'projeto', 'proposta', 'requerimento', 'destaque',
    'veto', 'câmara', 'senado', 'congresso', 'país', 'estado', 'brasil', 'lei', 'novo', 'nova',
    'política', 'povo', 'voto', 'partido', 'liderança', 'bancada', 'bloco', 'líder', 'lider', 'frente',
    'governo', 'oposição', 'presença', 'presente', 'passado', 'ausência', 'ausencia', 'ausente', 'obstrução',
    'registrar', 'aprovar', 'rejeitar', 'rejeição', 'sabe', 'matéria', 'materia', 'questão', 'ordem', 'emenda',
    'sistema', 'processo', 'legislativo', 'plenário', 'pedir', 'peço', 'comissão', 'especial', 'permanente',
    'apresentar', 'encaminhar', 'encaminho', 'orientar', 'liberar', 'apoiar', 'situação', 'fato', 'revisão',
    'tempo', 'pauta', 'discutir', 'discussão', 'debater', 'retirar', 'atender', 'colegas', 'autor', 'texto',
    'medida', 'união', 'república', 'audiência', 'audiencia', 'público', 'publico', 'reunião', 'agradecer',
    'solicitar', 'assistir', 'contrário', 'favorável', 'pessoa', 'comemorar', 'ato', 'momento', 'diretora',
    'possível', 'atenção', 'agradeço', 'naquele', 'necessárias', 'presidenta', 'compromisso', 'geradas',
    'primeiro', 'simplesmente', 'ideal', 'argumento', 'i', 'válido', 'envolvidos', 'nesse', 'aspecto',
    'existentes', 'normativo', 'irá', 'nada', 'melhor', 'esperarmos', 'pouco', 'resolvermos', 'problema',
    'postura', 'faltas', 'declara', '%', 'dia', 'obrigado', 'agradeço', 'agradecido', 'população',
    'maior', 'cada', 'bem', 'mundo', 'desta', 'mil', 'sendo', 'outros', '$', '!', '@', '#', '&', '(', ')',
    'r', 'sempre', 'além', 'semana', 'relação', 'onde', 'meio', 'inclusive', 'lá', 'vem', 'menos', 'menor',
    'qualquer', 'desde', 'ontem', 'hoje', 'exemplos', 'exemplo', 'tão', 'fim', 'janeiro', 'fevereiro', 'março',
    'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'alguns',
    'durante', 'gostaria', 'três', 'conta', 'feito', 'através', 'antes', 'depois', 'verdade', 'bom', 'quase',
    'setor', 'aí', 'disse', 'principalmente', 'final', 'vão', 'coisa', 'ver', 'sentido', 'vários',
    'nenhuma', 'quanto', 'infelizmente', 'felizmente', 'número', 'duas', 'dois', 'tanto', 'acho', 'achar',
    'enquanto', 'deve', 'apelo', 'papel', 'últimos', 'faço', 'fazer', 'garantir', 'garantia', 'fica',
    'obrigado..', 'assunto', 'sido', 'vir', 'incrementar', 'central', 'aproximado', 'aproximadamente',
    'hipotética', 'hipotese', 'hipótese', 'superiores', 'entende', 'pedido', 'oradora', 'tal', 'v.exas',
    'favor', 'vota', 'nº', 'srª', 'vista', 'sim', 'dito', 'tudo', 'obrigado', 'º', 'profundamente', 'custódio',
    'divulgado', 'características', 'perfeito', 'começarmos', 'nomes', 'amigo', 'possibilidade', 'mensagem',
    'come', 'parabenizar', 'começar', 'hs', 'atendimento', 'povos', '¯', 'ocorreu', 'entanto', 'diante',
    'defender', 'dr.', '“', '”', '•'
]


ONEGRAM_STOPWORDS = [
    'grande', 'nacional', 'são', 'e', 'de', 'das', 'dos', 'da', 'do', 'federal', 'cedo', 'urgência', 'equipe',
    'produtos', 'serviços', 'pequeno', 'total', 'podermos', 'consenso', 'popular', 'mérito', 'único', 'pública',
    'escolha', 'acesso', 'pilotos', 'trabalhar', 'ministério', 'países', 'combate', 'estados', 'vida', 'cidade',
    'municípios', 'histórico', 'defesa', 'município', 'prefeito', 'ii', 'santa', 'vereadora', 'centro',
    'prefeitura', 'governador', 'código', 'apoio', 'exercício', 'categoria', 'campo', 'kit'
]

def get_tokens(speeches, extra_stopwords=None):
    """
    Função que retorna tokens de discursos removendo as "stopwords".
    Argumentos:
        speeches: Recebe uma lista de discursos.
        stopwords: Recebe uma lista de palavras a serem retiradas dos textos.
    Retorna:
        Uma lista palavras do discurso que não estão nas "stopwords".
    """
    special_stopwords = ['são']
    stopwords = nltk_stopwords.words('portuguese') + list(punctuation) + EXTRA_STOPWORDS
    stopwords = [word for word in stopwords if word not in special_stopwords]
    if extra_stopwords:
        stopwords += extra_stopwords
    tokens = []
    for text in speeches:
        text = re.sub(r'[0-9]+', '', text)
        tokens += [i for i in word_tokenize(text.lower(), language='portuguese') if i not in stopwords]
    
    return tokens

tokens = get_tokens(speeches)
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
finder.score_ngrams(bigram_measures.likelihood_ratio)

[(('carlos', 'melles'), 800.2495073831722),
 (('minas', 'gerais'), 415.54149924400576),
 (('eduardo', 'cunha'), 411.1082108075898),
 (('carlos', 'manato'), 359.1476629536617),
 (('aécio', 'neves'), 337.09944421879146),
 (('manato', 'nobre'), 279.1314980290338),
 (('políticas', 'públicas'), 270.8304880207618),
 (('espírito', 'santo'), 263.94352472063485),
 (('são', 'paulo'), 258.6144269100274),
 (('fernando', 'coutinho'), 211.86201050118137),
 (('delegado', 'edson'), 210.42110870766425),
 (('jandira', 'feghali'), 210.42110870766425),
 (('joão', 'fernando'), 204.2895412867965),
 (('palmas', 'carlos'), 196.84321220369927),
 (('edson', 'moreira'), 195.40903600151864),
 (('segurança', 'pública'), 186.11291544157493),
 (('marcos', 'montes'), 178.70891580819412),
 (('policiais', 'militares'), 170.86734709018313),
 (('santa', 'catarina'), 168.93673407173785),
 (('evair', 'melo'), 163.74678430597345),
 (('mato', 'grosso'), 162.99506118224684),
 (('reforma', 'agrária'), 157.55976806867474),
 (('

In [28]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder2 = TrigramCollocationFinder.from_words(tokens)
finder2.score_ngrams(trigram_measures.likelihood_ratio)

[(('palmas', 'carlos', 'melles'), 1529.180449763135),
 (('carlos', 'melles', 'silas'), 1352.828332814156),
 (('carlos', 'melles', 'cumprimento'), 1248.6420179934983),
 (('carlos', 'melles', 'cooperativista'), 1227.776271521161),
 (('querido', 'carlos', 'melles'), 1219.010229218723),
 (('ajudar-nos', 'carlos', 'melles'), 1218.1998266935611),
 (('ajudaram', 'carlos', 'melles'), 1218.1998266935611),
 (('cafeeiras', 'carlos', 'melles'), 1218.1998266935611),
 (('carlos', 'melles', 'exmos'), 1218.1998266935611),
 (('carlos', 'melles', 'janio'), 1218.1998266935611),
 (('carlos', 'melles', 'louvável'), 1218.1998266935611),
 (('carlos', 'melles', 'muitíssimo'), 1218.1998266935611),
 (('carlos', 'melles', 'passa-se'), 1218.1998266935611),
 (('carlos', 'melles', 'piorou'), 1218.1998266935611),
 (('carlos', 'melles', 'registrou'), 1218.1998266935611),
 (('carlos', 'melles', 'salva'), 1218.1998266935611),
 (('falarmos', 'carlos', 'melles'), 1218.1998266935611),
 (('figuras', 'carlos', 'melles'), 12